In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
#load data
icu_data = pd.read_csv('icu/icustays_cleaned.csv')
diagnosis = pd.read_csv('ed/diagnosis_cleaned.csv')
triage = pd.read_csv('ed/triage_cleaned.csv')
vitals = pd.read_csv('ed/vitals_cleaned.csv')
ed_stays = pd.read_csv('ed/edstays_cleaned.csv')

# Combine the icd_version and icd_code columns in diagnosis table
diagnosis['icd_combined'] = diagnosis['icd_version'].astype(str) + '-' + diagnosis['icd_code'].astype(str)
print(diagnosis.head())

unique_icd_combined_count = diagnosis['icd_combined'].nunique()
print('Number of unique icd_combined:', unique_icd_combined_count)

#print(ed_stays.columns)
#print(icu_data.columns)


   subject_id   stay_id  seq_num icd_code  icd_version  \
0    10000032  32952584        1     4589            9   
1    10000032  32952584        2    07070            9   
2    10000032  32952584        3      V08            9   
3    10000032  39399961        1    78097            9   
4    10000032  39399961        2    34830            9   

                                           icd_title icd_combined  
0                                    HYPOTENSION NOS       9-4589  
1  UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...      9-07070  
2                         ASYMPTOMATIC HIV INFECTION        9-V08  
3                             ALTERED MENTAL STATUS       9-78097  
4                        ENCEPHALOPATHY, UNSPECIFIED      9-34830  
Number of unique icd_combined: 3894


In [5]:
# Function to calculate ranges for specified columns
def calculate_ranges(df, columns):
    ranges = {}
    for col in columns:
        col_data = df[col].replace(0.0, np.nan)  # Ignore values of 0
        ranges[col] = {
            'min': col_data.min(),
            'max': col_data.max()
        }
    return ranges

# Explore ranges in vitals and triage data
cols = ['heartrate', 'resprate', 'o2sat', 'sbp', 'dbp']
vitals_ranges = calculate_ranges(vitals, cols)
print("Ranges in vitals_cleaned:", vitals_ranges)

triage_ranges = calculate_ranges(triage, cols)
print("Ranges in triage_cleaned:", triage_ranges)

Ranges in vitals_cleaned: {'heartrate': {'min': np.float64(1.0), 'max': np.float64(705.0)}, 'resprate': {'min': np.float64(0.34), 'max': np.float64(78.0)}, 'o2sat': {'min': np.float64(1.0), 'max': np.float64(972.0)}, 'sbp': {'min': np.float64(2.0), 'max': np.float64(854.0)}, 'dbp': {'min': np.float64(2.0), 'max': np.float64(97100.0)}}
Ranges in triage_cleaned: {'heartrate': {'min': np.float64(14.0), 'max': np.float64(1228.0)}, 'resprate': {'min': np.float64(1.0), 'max': np.float64(189.0)}, 'o2sat': {'min': np.float64(2.0), 'max': np.float64(1004.0)}, 'sbp': {'min': np.float64(1.0), 'max': np.float64(9656.0)}, 'dbp': {'min': np.float64(4.0), 'max': np.float64(9102.0)}}


Following code links emergency department (ED) stays with ICU admissions and their corresponding length of stay (los). The resulting DataFrame is essential for downstream tasks like predictive modeling or analysis of ICU utilization based on ED visit data.

In [ ]:
# Merge datasets
merged_ed_icu = ed_stays[['stay_id', 'hadm_id']].merge(icu_data[['hadm_id', 'los']], on='hadm_id', how='inner')
merged_ed_icu = merged_ed_icu[['stay_id', 'hadm_id', 'los']]

print(merged_ed_icu.head())

    stay_id     hadm_id       los
0  32952584  29079034.0  0.410266
1  39399961  29079034.0  0.410266
2  30905710  26913865.0  0.497535
3  39866888  24597018.0  1.118032
4  34719194  26184834.0  9.171817


#### Processing Vitals Data
In this section, we process the `vitals` data to extract meaningful features that capture both static and dynamic aspects of a patient’s condition during their stay in the emergency department (ED). This approach ensures that the data is prepared for downstream tasks like predictive modeling or analysis. 

#### Why Process Vitals Data This Way?
- Static Aggregates for Overall Trends: The vital signs data (`heartrate`, `resprate`, `o2sat`, `sbp`, `dbp`) often includes multiple time-stamped measurements for each `stay_id`. Aggregating these values into a single `mean` value provides a concise summary of the patient's overall condition during their ED stay.
- Dynamic Trends for Detailed Insights: Aggregates like mean values can overlook significant temporal trends or changes in a patient’s condition. By calculating the slope of each vital sign over time (using linear regression), we capture these trends, such as:
    - Improvement: A patient’s oxygen saturation (o2sat) increases over time.
    - Deterioration: A patient’s blood pressure (sbp, dbp) decreases over time.
    
    These trends are critical for understanding how a patient’s condition evolves and can be strong predictors for downstream analysis (e.g., ICU length of stay).
- Combining Aggregates and Trends: Merging the static (mean, represente as _x) and dynamic (slope, represente as _slope) features ensures that both high-level and fine-grained patterns in the data are included for analysis or modeling.

#### Key Steps in the Code:
1. Aggregate Vitals Data: The `groupby` operation on `stay_id` aggregates vital sign measurements for each patient stay, providing static features like mean values for each vital sign.

2. Calculate Slope for Dynamic Trends: The `calculate_slope` function calculates the rate of change (slope) for each vital sign over time by performing linear regression on the time-stamped data (`charttime`) and the vital sign values.

3. Merge Vitals and Triage Data: Combines the `vitals_summary` data (static features) with the `triage` data using a left join on `stay_id`. `_x` refers to columns from the `triage` dataset. `_y` refers to columns from the `vitals_summary` dataset, which is the `mean` value.

4. Merge Static and Dynamic Features: After aggregating static values and calculating dynamic slopes, dynamic (slope) features are merged with other data to create a comprehensive feature set. 

5. Retain Desired Columns for Further Use: Only the most relevant columns (static aggregates, slopes, and identifiers) are retained for use in downstream analysis.

I think these can be write in the report.

Benefits of This Approach:
- Comprehensive Data Representation: Combining static and dynamic features captures both the overall condition and temporal patterns of a patient’s vitals.
- Improved Predictive Power: Trends (slopes) can highlight critical insights that static features alone cannot reveal, such as rapid deterioration or improvement in a patient’s condition.
- Cleaner, More Organized Dataset: Aggregating and merging the vitals data ensures that the resulting dataset is ready for machine learning models or statistical analysis.

In [ ]:
def calculate_slope(df, col):
    df = df.sort_values('charttime').dropna(subset=[col])  # Drop NaN rows
    time_delta = (pd.to_datetime(df['charttime']) - pd.to_datetime(df['charttime']).min()).dt.total_seconds()

    if len(time_delta.unique()) < 2 or df[col].nunique() < 2:
        return np.nan

    slope = np.polyfit(time_delta, df[col], 1)[0]
    return slope


# Aggregate vitals data
vitals_summary = vitals.groupby('stay_id').agg({
    'heartrate': 'mean',
    'resprate': 'mean',
    'o2sat': 'mean',
    'sbp': 'mean',
    'dbp': 'mean'
}).reset_index()

# Calculate slopes
slope_data = vitals.groupby('stay_id').apply(
    lambda x: pd.Series({
        'heartrate_slope': calculate_slope(x, 'heartrate'),
        'resprate_slope': calculate_slope(x, 'resprate'),
        'o2sat_slope': calculate_slope(x, 'o2sat'),
        'sbp_slope': calculate_slope(x, 'sbp'),
        'dbp_slope': calculate_slope(x, 'dbp')
    })
).reset_index()

# Merge triage and vitals summary
merged_features = triage.merge(vitals_summary, on='stay_id', how='left')

# Merge slopes into merged_features
merged_features = merged_features.merge(slope_data, on='stay_id', how='left')

# Retain only desired columns
ed_features = merged_features[[
    'stay_id', 'heartrate_x', 'resprate_x', 'o2sat_x', 'sbp_x', 'dbp_x',
    'heartrate_y', 'resprate_y', 'o2sat_y', 'sbp_y', 'dbp_y', 
    'heartrate_slope', 'resprate_slope', 'o2sat_slope', 'sbp_slope', 'dbp_slope'
]]

# Verify result
print(ed_features.head())


/var/folders/9p/f4lsl8xj05j2n9341rvs6ly40000gn/T/ipykernel_6928/909888495.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  time_delta = (pd.to_datetime(df['charttime']) - pd.to_datetime(df['charttime']).min()).dt.total_seconds()
/var/folders/9p/f4lsl8xj05j2n9341rvs6ly40000gn/T/ipykernel_6928/909888495.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  time_delta = (pd.to_datetime(df['charttime']) - pd.to_datetime(df['charttime']).min()).dt.total_seconds()
/var/folders/9p/f4lsl8xj05j2n9341rvs6ly40000gn/T/ipykernel_6928/909888495.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a 

    stay_id  heartrate_x  resprate_x  o2sat_x  sbp_x  dbp_x  heartrate_y  \
0  32952584         87.0        14.0     97.0   71.0   43.0    84.571429   
1  39399961         77.0        16.0     98.0   96.0   50.0    85.727273   
2  30905710         80.0        25.0     97.0  132.0   83.0    70.000000   
3  39866888         81.0        16.0     97.0  160.0  102.0    66.666667   
4  34719194         80.0        24.0     98.0  116.0   66.0    84.666667   

   resprate_y     o2sat_y       sbp_y      dbp_y  heartrate_slope  \
0   20.285714   98.000000   79.428571  42.857143         0.000133   
1   16.909091   94.363636   84.181818  50.545455         0.000785   
2   28.000000  100.000000  144.000000  87.000000        -0.001754   
3   16.000000   99.000000  143.666667  97.333333         0.000210   
4   21.166667   94.166667  135.600000  74.000000         0.000001   

   resprate_slope  o2sat_slope  sbp_slope  dbp_slope  
0       -0.000509     0.000099  -0.000282  -0.000404  
1       -0.000081 

/var/folders/9p/f4lsl8xj05j2n9341rvs6ly40000gn/T/ipykernel_6928/909888495.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  slope_data = vitals.groupby('stay_id').apply(


In [28]:
# should we merge heartrate_x and _y? or ignore that.



In [24]:
#Merge ED features with ICU and Diagnosis Data
final_data = pd.merge(ed_features, merged_ed_icu, on='stay_id', how='inner')
final_data = pd.merge(final_data, diagnosis[['stay_id', 'icd_combined']], on='stay_id', how='left')

# Verify result
print(final_data.head())

    stay_id  heartrate_x  resprate_x  o2sat_x  sbp_x  dbp_x  heartrate_y  \
0  32952584         87.0        14.0     97.0   71.0   43.0    84.571429   
1  32952584         87.0        14.0     97.0   71.0   43.0    84.571429   
2  32952584         87.0        14.0     97.0   71.0   43.0    84.571429   
3  39399961         77.0        16.0     98.0   96.0   50.0    85.727273   
4  39399961         77.0        16.0     98.0   96.0   50.0    85.727273   

   resprate_y    o2sat_y      sbp_y      dbp_y  heartrate_slope  \
0   20.285714  98.000000  79.428571  42.857143         0.000133   
1   20.285714  98.000000  79.428571  42.857143         0.000133   
2   20.285714  98.000000  79.428571  42.857143         0.000133   
3   16.909091  94.363636  84.181818  50.545455         0.000785   
4   16.909091  94.363636  84.181818  50.545455         0.000785   

   resprate_slope  o2sat_slope  sbp_slope  dbp_slope     hadm_id       los  \
0       -0.000509     0.000099  -0.000282  -0.000404  29079034

In [25]:
# Impute missing values
def impute_missing_values(df, numeric_features, non_numeric_features):
    # Impute numeric columns using the mean
    numeric_imputer = SimpleImputer(strategy='mean')
    df[numeric_features] = numeric_imputer.fit_transform(df[numeric_features])
    
    # Impute non-numeric columns with a constant value, e.g., 'unknown'
    non_numeric_imputer = SimpleImputer(strategy='constant', fill_value='unknown')
    df[non_numeric_features] = non_numeric_imputer.fit_transform(df[non_numeric_features])
    
    return df

# Define numeric and non-numeric features for imputation
numeric_features = ['heartrate_x', 'resprate_x', 'o2sat_x', 'sbp_x', 'dbp_x',
                    'heartrate_y', 'resprate_y', 'o2sat_y', 'sbp_y', 'dbp_y', 
                    'heartrate_slope', 'resprate_slope', 'o2sat_slope', 'sbp_slope', 'dbp_slope'
                    ]
non_numeric_features = ['icd_combined']

In [26]:
# Apply the imputation function
final_data = impute_missing_values(final_data, numeric_features, non_numeric_features)

# Function to calculate and handle outliers
def remove_outliers(df, col_ranges):
    for col_prefix, (min_val, max_val) in col_ranges.items():
        # Handle both '_x' and '_y' columns for each vital sign
        for suffix in ['_x', '_y']:
            col = col_prefix + suffix
            if col in df.columns:
                df[col] = np.where(df[col] < min_val, min_val, df[col])
                df[col] = np.where(df[col] > max_val, max_val, df[col])
    return df

# Define column ranges for vital signs
col_ranges = {
    'heartrate': (30, 180),
    'resprate': (8, 60),
    'o2sat': (50, 100),
    'sbp': (60, 250),
    'dbp': (30, 150),
}

# Apply outlier removal
final_data = remove_outliers(final_data, col_ranges)

# Verify result
print(final_data.head())

    stay_id  heartrate_x  resprate_x  o2sat_x  sbp_x  dbp_x  heartrate_y  \
0  32952584         87.0        14.0     97.0   71.0   43.0    84.571429   
1  32952584         87.0        14.0     97.0   71.0   43.0    84.571429   
2  32952584         87.0        14.0     97.0   71.0   43.0    84.571429   
3  39399961         77.0        16.0     98.0   96.0   50.0    85.727273   
4  39399961         77.0        16.0     98.0   96.0   50.0    85.727273   

   resprate_y    o2sat_y      sbp_y      dbp_y  heartrate_slope  \
0   20.285714  98.000000  79.428571  42.857143         0.000133   
1   20.285714  98.000000  79.428571  42.857143         0.000133   
2   20.285714  98.000000  79.428571  42.857143         0.000133   
3   16.909091  94.363636  84.181818  50.545455         0.000785   
4   16.909091  94.363636  84.181818  50.545455         0.000785   

   resprate_slope  o2sat_slope  sbp_slope  dbp_slope     hadm_id       los  \
0       -0.000509     0.000099  -0.000282  -0.000404  29079034

In [27]:
# Normalize data
id_columns = ['stay_id', 'hadm_id', 'icd_combined']
normalize_cols = ['heartrate_x', 'resprate_x', 'o2sat_x', 'sbp_x', 'dbp_x', 
                  'heartrate_y', 'resprate_y', 'o2sat_y', 'sbp_y', 'dbp_y', 
                  'heartrate_slope', 'resprate_slope', 'o2sat_slope', 'sbp_slope', 'dbp_slope']

# Separate data into two parts: identifiers and features to normalize
id_data = final_data[id_columns].copy()
features_to_normalize = final_data[normalize_cols].copy()

# Handle missing values in features before normalization
features_to_normalize.fillna(features_to_normalize.mean(), inplace=True)

# Normalize only the numeric columns
scaler = StandardScaler()
normalized_features = scaler.fit_transform(features_to_normalize)
normalized_features_df = pd.DataFrame(normalized_features, columns=normalize_cols)

# Concatenate the identifier columns with the normalized features
final_data_normalized = pd.concat([id_data.reset_index(drop=True), normalized_features_df.reset_index(drop=True)], axis=1)

# Handle missing values in non-numeric columns
final_data_normalized['icd_combined'] = final_data_normalized['icd_combined'].fillna('unknown')

# Save the processed data
final_data_normalized.to_csv('preprocessed_data.csv', index=False)
print("Final preprocessed data saved to 'preprocessed_data.csv'")




Final preprocessed data saved to 'preprocessed_data.csv'
